<a href="https://colab.research.google.com/github/PARTHIBAN-007/Transformers/blob/main/Causal_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install accelerate

  Using cached datasets-3.2.0-py3-none-any.whl.metadata (20 kB)
  Using cached evaluate-0.4.3-py3-none-any.whl.metadata (9.2 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 10.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4

In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [4]:
def any_keyword_in_String(string,keywords):
  for keyword in keywords:
    if keyword in string:
      return True
  return False


In [6]:
filters = ["pandas","sklearn","matplotlib","seaborn"]
example1 = "import numpy as np"
example2 = "import pandas as pd"

print(any_keyword_in_String(example1,filters))
print(any_keyword_in_String(example2,filters))

False
True


In [9]:
from collections import defaultdict
from tqdm import tqdm
from datasets import load_dataset

def filter_Streaming_dataset(dataset,filters):
  filtered_dict = defaultdict(list)
  total = 0
  for sample in tqdm(iter(dataset)):
    total +=1
    if any_keyword_in_String(sample["count"],filters):
      for k,v in sample.items():
        filtered_dict[k].append(v)
  print(f"{len(filtered_dict['content'])/total:.2%} of data after filtering")
  return Dataset.from_dict(filtered_dict)


In [12]:
from datasets import load_dataset , DatasetDict

ds_train = load_dataset("huggingface-course/codeparrot-ds-train", split="train")
ds_valid = load_dataset("huggingface-course/codeparrot-ds-valid", split="validation")

raw_dataset = DatasetDict(
    {
        "train" : ds_train,
        "valid" : ds_valid
    }
)
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['repo_name', 'path', 'copies', 'size', 'content', 'license'],
        num_rows: 606720
    })
    valid: Dataset({
        features: ['repo_name', 'path', 'copies', 'size', 'content', 'license'],
        num_rows: 3322
    })
})

In [16]:
for key in raw_dataset["train"][0]:
  print(f"{key.upper()} : {raw_dataset['train'][0][key][:200]}")

REPO_NAME : kmike/scikit-learn
PATH : sklearn/utils/__init__.py
COPIES : 3
SIZE : 10094
CONTENT : """
The :mod:`sklearn.utils` module includes various utilites.
"""

from collections import Sequence

import numpy as np
from scipy.sparse import issparse
import warnings

from .murmurhash import murm
LICENSE : bsd-3-clause


In [21]:
from transformers import AutoTokenizer
context_length = 128
tokenizer = AutoTokenizer.from_pretrained("huggingface-course/code-search-net-tokenizer")

outputs = tokenizer(
    raw_dataset["train"][:2]["content"],
    truncation = True ,
    max_length = context_length ,
    return_length = True,
    return_overflowing_tokens = True,
)

print(f"Input ID Length : {len(outputs['input_ids'])}")
print(f"Input chunk Length : {len(outputs['length'])}")
print(f"chunk Mapping : {len(outputs['overflow_to_sample_mapping'])}")

Input ID Length : 34
Input chunk Length : 34
chunk Mapping : 34


In [ ]:
def tokenize(element):
  outputs = tokenizer(
      element["content"],
      truncation =True ,
      max_length = context_length,
      return_overflowing_tokens = True,
      return_length = True,
  )

  input_batch = []
  for length , input_ids in zip(outputs["length"],outputs["input_ids"]):
    if length == context_length :
      input_batch.append(input_ids)
  return {"input_ids" : input_batch}


tokenized_dataset = raw_dataset.map(
    tokenize, batched =True ,remove_columns = raw_dataset["train"].column_names
)
tokenized_dataset

Map:   0%|          | 0/606720 [00:00<?, ? examples/s]

In [ ]:
from transformers import GPT2LMHeadModel, AutoConfig

config = AutoConfig.from_pretrained(
    "gpt2",
    vocab_size = len(tokenizer),
    n_ctx = context_length ,
    bos_token_id = tokenizer.bos_token_id ,
    eos_token_id = tokenizer.eos_token_id,
)


In [ ]:
model = GPT2LMHeadModel(config)
model_size = sum(t.numel() for t in model.parameters())
print(f"GPT-2 size : {model_size/1000**2:.1f}M parameters")